In [2]:
import os
import json
from datetime import datetime
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as plt
import dask
import graphviz
import time




## EODC and dask

https://github.com/eodcgmbh/eodc-examples/blob/main/demos/dask.ipynb

To use dask on the external servers of EODC we have to follow the tutorial from above. In order to have access to these servers we have to have a username. 